In [29]:
import lief
import pefile
from typing import cast
import shutil
import lief
import polars as pl

In [30]:
path = r"G:\Games\FA\FA-EMU\Shipping\GAME_dump.exe"
original_path = r"G:\Games\FA\FA-EMU\Shipping\GAME.exe"
out_path = r"G:\Games\FA\FA-EMU\Shipping\GAME_dump_mod.exe"

shutil.copy(path, out_path)

'G:\\Games\\FA\\FA-EMU\\Shipping\\GAME_dump_mod.exe'

In [31]:
# pe = pefile.PE(out_path, fast_load=True)
# pe.full_load()

pe = lief.PE.parse(out_path)

In [32]:
def remove(file: pefile.PE, index: int):
    """Removes a section of the section table.
       Deletes the section header in the section table, the data of the section
       in the file, removes the section in the sections list of pefile and adjusts
       the sizes in the optional header.
    """

    # Checking if the section list is long enough to actually remove index.
    if (file.FILE_HEADER.NumberOfSections > index
        and file.FILE_HEADER.NumberOfSections == len(file.sections)):

        # Stripping the data of the section from the file.
        if file.sections[index].SizeOfRawData != 0:
            file.__data__ = \
                (file.__data__[:file.sections[index].PointerToRawData] +
                 file.__data__[file.sections[index].PointerToRawData +
                 file.sections[index].SizeOfRawData:])

        # Overwriting the section header in the binary with nulls.
        # Getting the address of the section table and manually overwriting
        # the header with nulls unfortunally didn't work out.
        file.sections[index].Name = '\x00'*8
        file.sections[index].Misc_VirtualSize = 0x00000000
        file.sections[index].VirtualAddress = 0x00000000
        file.sections[index].SizeOfRawData = 0x00000000
        file.sections[index].PointerToRawData = 0x00000000
        file.sections[index].PointerToRelocations = 0x00000000
        file.sections[index].PointerToLinenumbers = 0x00000000
        file.sections[index].NumberOfRelocations = 0x0000
        file.sections[index].NumberOfLinenumbers = 0x0000
        file.sections[index].Characteristics = 0x00000000

        del file.sections[index]

In [33]:
bad_section_name = [
    # 'wlovhtaq',  # DON'T DELETE THAT! TLS Directory here
    # 'oemvvlbu',
]

if isinstance(pe, pefile.PE):
    for name in bad_section_name:
        sect_i = 0
        for i, sect in enumerate(pe.sections):
            if sect.Name.decode() == name:
                sect_i = i
                break
        else:
            print(f"Failed to locate {name} section")
            continue

        remove(pe, sect_i)
        print(f"Removed {name}:{sect_i} section")
        
if isinstance(pe, lief.PE.Binary):
    for name in bad_section_name:
        pe.remove_section(name, clear=True)
        print(f"Removed {name} section")

In [34]:
pe.remove_all_imports()

In [35]:
base = '../game-dump/'
fn_thunks = base + "thunks.csv"
thunks = pl.read_csv(fn_thunks)

In [36]:
def create_32bit_ordinal_import(ordinal_number: int) -> lief.PE.ImportEntry:
    """
    Create a 32-bit import by ordinal
    
    Args:
        ordinal_number: The ordinal number (0-65535)
    """
    # Validate ordinal range
    if ordinal_number < 0 or ordinal_number > 0xFFFF:
        raise ValueError("Ordinal number must be between 0 and 65535")
    
    # For 32-bit PE:
    # - Set bit 31 to 1 (0x80000000)
    # - Bits 30-16 must be 0
    # - Bits 15-0 contain the ordinal
    ORDINAL_MASK_32 = 0x80000000
    data_value = ORDINAL_MASK_32 | ordinal_number
    
    # Create the import entry
    entry = lief.PE.ImportEntry(data_value, lief.PE.PE_TYPE.PE32)
    
    return entry

In [37]:
dlls = thunks['Module'].unique()
for dll in dlls:
    mod = pe.add_import(dll)
    imports = thunks.filter(pl.col("Module") == dll)["Function"]
    imports_addr = thunks.filter(pl.col("Module") == dll)["Call address"]
    for entry, addr in zip(imports, imports_addr):
        if entry.startswith("Ordinal#"):
            ordinal = int(entry.removeprefix('Ordinal#'))
            print('made by ordinal', ordinal, mod.add_entry(create_32bit_ordinal_import(ordinal)))
        else:
            mod.add_entry(entry)

made by ordinal 1 0x0000: 1
made by ordinal 2 0x0000: 2


In [38]:
config = lief.PE.Builder.config_t()
config.imports = True

bb = lief.PE.Builder(pe, config)
bb.build()
bb.write(out_path)

Can't find section with the rva: 0x0


## Troubleshooting

In [39]:
print(f"Subsystem: {pe.optional_header.subsystem}")
print(f"Characteristics: {hex(pe.header.characteristics)}")
print(f"DLL Characteristics: {hex(pe.optional_header.dll_characteristics)}")

Subsystem: SUBSYSTEM.WINDOWS_GUI
Characteristics: 0x123
DLL Characteristics: 0x8000


In [40]:
print(f"Entry Point: 0x{pe.optional_header.addressof_entrypoint:x}")
print(f"Image Base: 0x{pe.optional_header.imagebase:x}")

Entry Point: 0x10ab273
Image Base: 0x400000


In [41]:
if pe.has_tls:
    tls = pe.tls
    print(f"TLS Callbacks: {[hex(cb) for cb in tls.callbacks]}")

TLS Callbacks: []


In [42]:
original_pe = lief.PE.parse(original_path)

print(f"Subsystem: {original_pe.optional_header.subsystem}")
print(f"Characteristics: {hex(original_pe.header.characteristics)}")
print(f"DLL Characteristics: {hex(original_pe.optional_header.dll_characteristics)}")

if original_pe.has_tls:
    tls = original_pe.tls
    print(f"TLS Callbacks: {[hex(cb) for cb in tls.callbacks]}")

Subsystem: SUBSYSTEM.WINDOWS_GUI
Characteristics: 0x123
DLL Characteristics: 0x8000
TLS Callbacks: []


TLS's template corrupted
